## Lesson S2 - WRF Preprocessing System (WPS)
## Overview
In this lesson, we will cover regridding NLDAS2 forcing data to a WRF-Hydro domain using the ESMF
regridding functions within the NCAR Command Language (NCL).

**NOTE: If you have not completed lesson S1 please stop and do so now**

### Software and conventions
The easiest way to run these lessons is via the [wrfhydro/training](https://hub.docker.com/r/wrfhydro/training/) Docker container, which has all software dependencies and data pre-installed.

For a complete description of the software environment used for this training please see [Getting started]().

You may either execute commands by running each cell of this notebook. Alternatively, you may open a terminal in Jupyter Lab by selecting `New -> Terminal` in your `Home` tab of Jupyter Lab and input the commands manually if you prefer. You can also use your own terminal by logging into the container with the command `docker exec -it wrf-hydro-training bash`

All paths used in this lesson assume that the lesson materials are located under your home directory in a folder named `wrf-hydro-training`. If your materials are located in another directory, you will not be able to run the commands in this notebook inside Jupyter and will need to type them manually in your terminal session. 

## Required scripts and datasets

### Regridding scripts
Scripts and documentation to regrid NLDAS2 forcing data can be downloaded from the WRF-Hydro website at https://ral.ucar.edu/projects/wrf_hydro/regridding-scripts.

### Data sources
For this training, data has already been included for simulation time period. 

**NOTE: These data are for instructional purposes only and do not include forcing data used for model spinup**

If you would like to acquire other data, NLDAS2 forcing data can be retrieved from the NASA data server at https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_FORA0125_H.002/. 

See the following site for information on how to access data:
https://disc.gsfc.nasa.gov/data-access 

See the following instructions for how to download data files using wget:
https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Download%20Data%20Files%20from%20HTTP%20Service%20with%20wget

## Creating the directory structure
The NCL regridding scripts require a **very** specific directory structure, which is outlined below:

```
|
input_files
    |_NLDAS_FORA0125_H.*.grb - All the NLDAS2 input forcing data
|_NLDAS2WRFHydro_generate_weights.ncl - NCL script for generating weight file
|_NLDAS2WRFHydro_regrid.ncl - NCL script for regridding
```

In this section, we will create this directory structure
**Step 1: Extract the NLDAS2 forcing data provided with this tutorial**

In [61]:
%%bash
cd ~/wrf-hydro-training/regridding/data
tar -xf ~/wrf-hydro-training/regridding/data/nldas_forcing.tar.gz

**Step 2: Create an *input_files* directory to hold the forcing data and move the data to it.**

In [62]:
%%bash
mkdir -p mkdir ~/wrf-hydro-training/output/lessonS2/input_files
mv ~/wrf-hydro-training/regridding/data/nldas_forcing/*.grb \
~/wrf-hydro-training/output/lessonS2/input_files

**Step 3: View the contents of the input_files directory files**

In [63]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2/input_files | head -10

NLDAS_FORA0125_H.A20110826.0000.002.grb
NLDAS_FORA0125_H.A20110826.0100.002.grb
NLDAS_FORA0125_H.A20110826.0200.002.grb
NLDAS_FORA0125_H.A20110826.0300.002.grb
NLDAS_FORA0125_H.A20110826.0400.002.grb
NLDAS_FORA0125_H.A20110826.0500.002.grb
NLDAS_FORA0125_H.A20110826.0600.002.grb
NLDAS_FORA0125_H.A20110826.0700.002.grb
NLDAS_FORA0125_H.A20110826.0800.002.grb
NLDAS_FORA0125_H.A20110826.0900.002.grb


**Step 4: View the contents of the input_files directory files**
Bring in the ncl scripts

In [64]:
%%bash
cp ~/wrf-hydro-training/regridding/*.ncl \
~/wrf-hydro-training/output/lessonS2/

**Step 5: Check that your directory structure is correct** 

In [65]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2/

NLDAS2WRFHydro_generate_weights.ncl
NLDAS2WRFHydro_regrid.ncl
input_files


Our directory structure is established, now we can begin the regridding process.

## Creating the regridding weight file
The weight files are netCDF files which specify interpolation weights between the source coordinate data grids and destination coordinate data grids. 

We will use the script *NLDAS2WRFHydro_generate_weights.ncl* to generate the weight file. This script takes a geogrid file and a single NLDAS2 forcing *.grb* file as inputs and returns a weight file.

**Step 1: Creating the regridding weight file**
We will use the *NLDAS2WRFHydro_generate_weights.ncl* script to create the weight file

Recall that we created a geogrid file in Lesson S1. We will use this geogrid file as input to the regridding script along with a single NLDAS2 forcing file. We will run the NCL scripts from our output directory because outputs are written to the current working directory

In [66]:
%%bash
cd ~/wrf-hydro-training/output/lessonS2
ncl 'interp_opt="bilinear"' \
    'srcGridName="./input_files/NLDAS_FORA0125_H.A20110826.0000.002.grb"' \
    'dstGridName="~/wrf-hydro-training/output/lessonS1/geo_em.d01.nc"' \
    NLDAS2WRFHydro_generate_weights.ncl

 Copyright (C) 1995-2017 - All Rights Reserved
 University Corporation for Atmospheric Research
 NCAR Command Language Version 6.4.0
 The use of this software is governed by a License Agreement.
 See http://www.ncl.ucar.edu/ for more details.
(0)	=====> CPU Elapsed Time: rectilinear_to_SCRIP: 0.06 seconds <=====
(0)	=====> CPU Elapsed Time: curvilinear_to_SCRIP: 0.00999999 seconds <=====
(0)	ESMF_regrid_gen_weights: number of processors used: 1
(0)	--------------------------------------------------
(0)	ESMF_regrid_gen_weights: the following command is about to be executed on the system:
(0)	'ESMF_RegridWeightGen --ignore_degenerate  --source SCRIP_NLDAS_bilinear.nc --destination SCRIP_WRFHydro_bilinear.nc --weight ./NLDAS2WRFHydro_weight_bilinear.nc --method bilinear --dst_regional -i'
(0)	--------------------------------------------------
(0)	ESMF_regrid_gen_weights: output from 'ESMF_RegridWeightGen':
(0)	      Starting weight generation with these inputs: 
(1)	        Source File: S

ncl: /home/docker/miniconda3/bin/../lib/./libuuid.so.1: no version information available (required by /home/docker/miniconda3/bin/../lib/./libdapserver.so.7)


In [67]:
%%bash
ls ~/wrf-hydro-training/output/lessonS2

NLDAS2WRFHydro_generate_weights.ncl
NLDAS2WRFHydro_regrid.ncl
NLDAS2WRFHydro_weight_bilinear.nc
PET0.RegridWeightGen.Log
SCRIP_NLDAS_bilinear.nc
SCRIP_WRFHydro_bilinear.nc
input_files


We have two new files: The *SCRIP_NLDAS_bilinear.nc* describes the NLDAS grids and the *SCRIP_WRFHydro_bilinear.nc* describes the WRF-Hydro grid from the geogrid file.

## Regridding the forcing data using the weight file
In this section we will regrid all our forcing data using the weight file generated in the previous section. 

**Step 2: Run the *NLDAS2WRFHydro_regrid.ncl* regridding script**

The *NLDAS2WRFHydro_regrid.ncl* script takes 2 arguments, the path to the geogrid file and the path to NLDAS2 forcing data. **NOTE: Wildcards can be used in the path to the forcing data to regrid multiple files.**

In [69]:
%%bash
cd ~/wrf-hydro-training/output/lessonS2
ncl 'srcFileName="NLDAS_FORA0125_H.*.grb"' \
'dstGridName="~/wrf-hydro-training/output/lessonS1/geo_em.d01.nc"' \
NLDAS2WRFHydro_regrid.ncl &> regrid.log